In [1]:
# Laura Chapman
# Script for developing statistical tools for Glue
# Computes statistics for subsets as well as entire data using compute_statistic
# Changed from astropy tables to pandas dataframe

# Format data well in a popup using qt
# Color code by subset

In [2]:
# Basic code that imports glue and loads in and links the data

import sys
from glue.core.data_factories import load_data
from glue.core import DataCollection
from glue.core.link_helpers import LinkSame
from glue.app.qt.application import GlueApplication
from glue.viewers.image.qt import ImageViewer
from glue_vispy_viewers.volume.volume_viewer import VispyVolumeViewer

image_filename='w5.fits'
catalog_filename='w5_psc.vot'

#load 2 datasets from files
catalog = load_data(catalog_filename)
image = load_data(image_filename)

dc = DataCollection([catalog,image])

# link positional information
dc.add_link(LinkSame(catalog.id['RAJ2000'], image.id['Right Ascension']))
dc.add_link(LinkSame(catalog.id['DEJ2000'], image.id['Declination']))

#Create subset based on filament mask
ra_state=(image.id['Right Ascension'] > 44) & (image.id['Right Ascension'] < 46)
subset_group=dc.new_subset_group('RA_Selection',ra_state)
subset_group.style.color = '#0000FF'



In [3]:
import glue.utils.array as gua
import glue.core.data as gcd
from astropy.table import Table

In [4]:
# Constructs a pandas DataFrame instead of an astropy table

import pandas as pd
from pandas import DataFrame
import numpy as np

In [5]:
# Defines a subset and runs statistics using compute_statistic for the subset
# Saves data using a pandas dataframe called my_pandas_data

# Data that subset is pulled from
data = dc[0]

# Define state and subset
state1 = data.id['Jmag'] > 14
subset1 = data.new_subset(state1, label='Jmag > 14')

# Same arrays as with full data
mean_array = []
median_array = []
min_array = []
max_array = []
sum_array = []
name_array = []
tables = []

headings = ('mean', 'median', 'minimum', 'maximum', 'sum')

print(data.label, 'subset1')
print() 
for j in range (0, len(data.components)):
    name = data.components[j].label # Get the name of each component
    name_array.append(name) # add to the name array to build the table
    mean_array.append(data.compute_statistic('mean', subset1.components[j], subset_state=subset1.subset_state))
    median_array.append(data.compute_statistic('median', subset1.components[j], subset_state=subset1.subset_state))       
    min_array.append(data.compute_statistic('minimum', subset1.components[j], subset_state=subset1.subset_state))       
    max_array.append(data.compute_statistic('maximum', subset1.components[j], subset_state=subset1.subset_state))      
    sum_array.append(data.compute_statistic('sum', subset1.components[j], subset_state=subset1.subset_state))        
   
column_data = np.asarray([mean_array, median_array, min_array, max_array, sum_array]).transpose()

my_pandas_data = pd.DataFrame(column_data, index=name_array, columns=headings)

w5_psc subset1



In [8]:
from PyQt5.QtCore import QAbstractItemModel, QFile, QIODevice, QModelIndex, Qt
from PyQt5.QtWidgets import QApplication, QTreeView, QAbstractItemView
from PyQt5.QtWidgets import QApplication, QLabel, QTreeView, QComboBox, QWidget, QPushButton, QHBoxLayout, QFrame, QTableView,QGroupBox, QDialog, QVBoxLayout, QLabel,QGridLayout
from PyQt5 import QtCore, QtWidgets, QtGui
from PyQt5.QtGui import QStandardItemModel, QStandardItem
from PyQt5.QtCore import QItemSelectionModel
from PyQt5.QtCore import pyqtSlot, QVariant

import sys

class pandasModel(QtCore.QAbstractTableModel):
    # Set up the data in a form that allows it to be added to qt widget
    def __init__(self, df, parent=None):
        QtCore.QAbstractTableModel.__init__(self, parent)
        self.data_frame = df
        super(pandasModel, self).__init__(parent)      

    def rowCount(self, parent=None):
        return len(self.data_frame.values)

    def columnCount(self, parent=None):
        return self.data_frame.columns.size

    def data(self, index, role=Qt.DisplayRole):
        if index.isValid():
            if role == Qt.DisplayRole:
                return QVariant(str(
                    self.data_frame.values[index.row()][index.column()]))
        return QVariant()
    
    def headerData(self, col, orientation, role):
        if orientation == Qt.Horizontal and role == Qt.DisplayRole:
            return QVariant(self.data_frame.columns[col])
        if orientation == Qt.Vertical and role == Qt.DisplayRole:
            return QVariant(self.data_frame.index[col])
        return QVariant()  

class StatsGui(QWidget):
    ''' 
    This class accepts a glue data collection object, and builds an interactive window
    to display basic statistics (e.g. mean, median, mode) about each dataset
    '''
    def __init__(self,dc):
        
        # Initialize the object as a QWidget
        QWidget.__init__(self)
    
        #Save the datacollection object as an attribute of class StatsGui
        self.dc=dc

        #Fix the size of the main GUI window (for now)
        self.setMinimumSize(800, 600)
        self.setMaximumSize(800, 600)
        
        #Set the title of the main GUI window
        self.setWindowTitle('Statistics')
        
        #Set up tree view and fix it to the top half of the window
        self.treeview = QTreeView(self)
        self.treeview.setGeometry(50, 25, 700, 200) 
        
        #Every time a selection is made in the tree, perform the functionality in 'myPressedEvent'
        self.treeview.pressed.connect(self.myPressedEvent)

        #Set the default clicking behavior to be row selection
        self.treeview.setSelectionBehavior(QAbstractItemView.SelectRows)
        model = QStandardItemModel()
        model.setHorizontalHeaderLabels([''])

        self.treeview.setModel(model)
        self.treeview.setUniformRowHeights(True)
        
        #Allow the user to select multiple rows at a time 
        self.treeview.setSelectionMode(QAbstractItemView.MultiSelection)
    
        # populate the tree
        # Make all the datasets be parents, and make it so they are not selectable
        for i in range(0,len(dc)):
            parent = QStandardItem('{}'.format(self.dc.labels[i]))
            parent.setEditable(False)
            parent.setSelectable(False)
            
            # Make all the data components be children, nested under their parent
            for j in range(0,len(self.dc[i].components)):
                child=QStandardItem('{}'.format(str(self.dc[i].components[j])))
                child.setEditable(False)
                parent.appendRow(child)
            
            #Add the parents with their children to the QStandardItemModel
            model.appendRow(parent)

        #################Set up the QTableView Widget#############################
        self.table = QTableView(self)
        
        #Move the table widget to the bottom of the GUI window
        self.table.setGeometry(50, 250, 700, 300) 
        
        #Set the table headings -- I have added Dataset and Component Columns. 
        self.headings = ('Dataset','Component','Mean', 'Median', 'Minimum', 'Maximum', 'Sum')   
        self.data_frame = pd.DataFrame(columns=self.headings)                      
        self.model = pandasModel(self.data_frame)

        self.table.setModel(self.model)
        self.table.setShowGrid(False)  
        
    def myPressedEvent (self, currentQModelIndex):
        ''' 
        Every time a row (or rows) in the tree view is clicked or unclicked, this function
        returns the dataset index and the component index for all selected rows.
        You should be able to use this info determine the current table view
        ''' 

        indexes=self.treeview.selectionModel().selectedRows()
        for index in sorted(indexes):
            print('Row %d in dataset %d is selected' % (index.row(),index.parent().row()))
        
        print('~~~~~~~')
        
app = QApplication.instance()
if app is None:
    app = QApplication(sys.argv)
else:
    print('QApplication instance already exists: %s' % str(app))
ex = StatsGui(dc)
ex.show()
sys.exit(app.exec_())

QApplication instance already exists: <PyQt5.QtWidgets.QApplication object at 0x151a2d3438>
Row 2 in dataset 1 is selected
~~~~~~~
Row 1 in dataset 1 is selected
Row 2 in dataset 1 is selected
~~~~~~~


SystemExit: 0

/anaconda3/envs/glueviz-dev/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
